## Decision Tree Learning
- Decision 1: How to choose what feature to split on at each node to maximize purity (minimize impurity)
- Decision 2: When do you stop splitting?
    - When a node is 100% one class
    - WHen splitting a node will result in exceeding a maximum depth
    - When improvements in purity score are below a threshold
    - When the number of examples in a node is below a threshold

## Entropy
- measure of impirity of data
- Entropy function: H(p1 )
- closer to 50/50 mix, closer to maximum entropy (1.0)

**H(p1) = -p1*log2(p1) - p0*log2(p0)** = -p1*log2(p1) - (1-p1)*log2(-p1)

## Choosing a Split: Information Gain
- node with lots of examples and high entropy is worse than node with few examples and high entropy
- **Information Gain:** the reduction in entropy that you get from your tree resulting from making a split
    - p1_left = fraction in left split with a positive label
    - w_left = fraction of examples out of all root examples that went to left node

    - p1_right = fraction in right split with a positive label
    - w_right = fraction of examples out of all root examples that went to right node

    - p1_root = fraction of positive labels in root node

***Information Gain = H(p1_root) - [w_left * H(p1_left) + w_right * H(p1_right)]***

- for regression trees, p1_root, p1_left and p1_right are **varainces** -> split on biggest variance reduction

## Decision Tree Code

In [4]:
import numpy as np

# load some examples. X_train contains 3 features: ear shape (1 = points), face shape (1 = round), and whiskers (1 = present)
# y_train contains labels, 1 = cat, 0 = otherwise
X_train = np.array([[1, 1, 1],
[0, 0, 1],
 [0, 1, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 1, 0],
 [0, 0, 0],
 [1, 1, 0],
 [0, 1, 0],
 [0, 1, 0]])

y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [5]:
# entropy function
def entropy(p):
    # handels boundary conditions where function would be mathematically undefined
    if p == 1 or p == 1:
        return 0
    else:
        return -p * np.log2(p) - (1-p) * np.log2(1-p)
    

In [6]:
def split_indices(X, index_feature):
    left_indicies = []
    right_indicies = []

    for i, x in enumerate(X):
        if x[index_feature] == 1:
            left_indicies.append(i)
        else:
            right_indicies.append(i)
    
    return left_indicies, right_indicies

In [11]:
def weighted_entropy(X, y, left_indicies, right_indicies):
    
    # proportion of examples at node
    w_left = len(left_indicies) / len(X)
    w_right = len(right_indicies) / len(X)

    # proportion of labels in node
    p_left = sum(y[left_indicies]) / len(left_indicies)
    p_right = sum(y[right_indicies]) / len(right_indicies)

    weighted_entropy = w_left * entropy(p_left) + w_right * entropy(p_right)

    return weighted_entropy

In [12]:
def information_gain(X, y, left_indicies, right_indicies):
    p_node = sum(y) / len(y)
    h_node = entropy(p_node)
    w_entropy = weighted_entropy(X, y, left_indicies, right_indicies)

    info_gain = h_node - w_entropy
    
    return info_gain

In [13]:
# information gain for splitting root for each feature

for i, feature_name in enumerate(['Ear Shape', 'Face Shape', 'Whiskers']):
    left_indices, right_indices = split_indices(X_train, i)
    i_gain = information_gain(X_train, y_train, left_indices, right_indices)
    print(f"Feature: {feature_name}, information gain if we split the root node using this feature: {i_gain:.2f}")

Feature: Ear Shape, information gain if we split the root node using this feature: 0.28
Feature: Face Shape, information gain if we split the root node using this feature: 0.03
Feature: Whiskers, information gain if we split the root node using this feature: 0.12


## Tree Ensembles ##

- majority vote amonst the trees gives overall result
- create new random training set using sampling with replacement (can include repeats)

Procedure - Bagged Trees
given training set of size m
for b = 1 to B:
    use sampling with replacement to create a new training set of size m
    train a decision tree on the new data set

- **Reccommended 64-128 trees**. Larger B never hurts performance but diminishing returns


Random Forest Algorithm
- At each node, when choosing a feature to use to split, if n features are available, pick a random subset of k < features and allow the algorithm to only choose from that subset of features
    - typically k = √n

##  (eXtreme Gradient) Boost
- modification to bagged decision tree algorithm

given training set of size m
for b = 1 to B:
    use sampling with replacement to create a new training set of size m, **but instead of picking from all examples with equal (1/m) probability, make it more likely to pick misclassified examples from previously trained trees
    train a decision tree on the new data set



**Benefits of XG boost:**
- open source implementation of boosted trees
- fast efficient implementation
- good choice of default splitting criteria and criteria for when to stop splitting
- built in regularization to prevent overfitting
- highly competitive algorithm for machine learning competitions (i.e Kaggle competitions)
